In [5]:
import os
from kiwillama import interface, settings, train, utils

gpus = '1,2'
os.environ['CUDA_VISIBLE_DEVICES'] = gpus

model_name = './models/Llama-3-8b-mix-2epoch/merged/'

device_map = [f"cuda:{i}" for i in gpus.split(",")]

from vllm import LLM,SamplingParams
sampling_params = SamplingParams(max_tokens=512,stop='<EOS>',temperature=0)
llm = LLM(model=f"{model_name}", tensor_parallel_size=len(device_map))  # Create an LLM.

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


Generating train split: 0 examples [00:00, ? examples/s]

OSError: There was a specific connection error when trying to load meta-llama/Meta-Llama-3-8B-Instruct:
401 Client Error: Unauthorized for url: https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json (Request ID: Root=1-67c253e6-4aca3abc5cc7e31507d1ac58;978ccda7-d16d-4cbd-8b43-26e88d1a5e99)

Invalid credentials in Authorization header

In [4]:
import os
from glob import glob
prompt = '''### Task:
Your task is to generate an HTML version of an input text, using HTML <span> tags to mark up specific entities.

### Entity Markup Guides:
Use <span class="problem"> to denote a medical problem.
Use <span class="treatment"> to denote a treatment.
Use <span class="test"> to denote a test.
Use <span class="drug"> to denote a drug.

### Entity Definitions:
Medical Problem: The abnormal condition that happens physically or mentally to a patient.
Treatment: The procedures, interventions, and substances given to a patient for treating a problem.
Drug: Generic or brand name of a single medication or a collective name of a group of medication.
Test: A medical procedure performed (i) to detect or diagnose a problem, (ii) to monitor diseases, disease processes, and susceptibility, or (iii) to determine a course of treatment.

### Input Text: {} <EOS>
### Output Text:'''



In [3]:
def batch_list(input_list, batch_size):
    batched_list = []
    for i in range(0, len(input_list), batch_size):
        batched_list.append(input_list[i:i + batch_size])
    return batched_list

In [4]:
from pynvml import *
import time

def get_gpu_memory_usage():
    nvmlInit()
    device_count = nvmlDeviceGetCount()  # Get number of GPUs
    memory_usage = []
    for i in [5,6]:
        handle = nvmlDeviceGetHandleByIndex(i)
        info = nvmlDeviceGetMemoryInfo(handle)
        memory_usage.append(info.used / 1024 ** 2)  # Convert to MB
    return memory_usage

def get_gpu_power_usage():
    device_count = nvmlDeviceGetCount()  # Get number of GPUs
    power_usage = []
    for i in [5,6]:
        handle = nvmlDeviceGetHandleByIndex(i)
        power = nvmlDeviceGetPowerUsage(handle)  # Returns power in milliwatts
        power_usage.append(power / 1000)  # Convert to watts
    return power_usage


In [5]:
import spacy
import pandas as pd
import re

def group_relations(tuples_set):
    grouped_tuples = {}
    for pair in tuples_set:
        key = pair[0]  # The first tuple of the pair
        if key in grouped_tuples:
            grouped_tuples[key].append(pair[1])
        else:
            grouped_tuples[key] = [pair[1]]
            
    # Sorting the values in each group in descending order based on the second value of the tuple
    sorted_grouped_tuples = {}
    for key, values in grouped_tuples.items():
        sorted_grouped_tuples[key] = sorted(values, key=lambda x: x[1], reverse=True)

    return sorted_grouped_tuples

def replace_entities_with_types(sent, entities):
    sent_text = str(sent)
    if isinstance(entities, list):
        for e in entities:
            ent_type, start, end =e
            sent_text = sent_text[:start - sent.start_char]+f'<span class="{ent_type}">{sent_text[start - sent.start_char:end - sent.start_char]}</span>'+sent_text[end - sent.start_char:] 
    else:
        ent_type, start, end =entities
        sent_text = sent_text[:start - sent.start_char]+f'<span class="{ent_type}">{sent_text[start - sent.start_char:end - sent.start_char]}</span>'+sent_text[end - sent.start_char:] 
    return sent_text

def sentence_relations(text,entities, relations, nlp):
    test_prompt = '''### Task:
    Your task is to mark up modifier entities related to the entity marked with <span> tag in the input text.
    
    ### Entity Markup Guide:
    Use <span class="labvalue"> to denote a numeric value or a normal description of the result of a lab test.
    Use <span class="reference_range"> to denote the range or interval of values that are deemed as normal for a test in a healthy person.
    Use <span class="negation"> to denote the phrase that indicates the absence of an entity.
    Use <span class="temporal"> to denote a calendar date, time, or duration related to a test.

    ### Input Text: {} <EOS>
    ### Output Text:'''
    
    drug_prompt = '''### Task:
    Your task is to mark up modifier entities related to the entity marked with <span> tag in the input text.

    ### Entity Markup Guide:
    Use <span class="form"> to denote the form of drug.
    Use <span class="frequency"> to denote the frequency of taking a drug.
    Use <span class="dosage"> to denote the amount of active ingredient from the number of drugs prescribed.
    Use <span class="duration"> to denote the time period a patient should take a drug.
    Use <span class="strength"> to denote the amount of active ingredient in a given dosage form.
    Use <span class="route"> to denote the way by which a drug, fluid, poison, or other substance is taken into the body.
    Use <span class="negation"> to denote the phrase that indicates the absence of an entity.
    Use <span class="temporal"> to denote a calendar date, time, or duration related to a drug.

    ### Input Text: {} <EOS>
    ### Output Text:'''

    problem_prompt = '''### Task:
    Your task is to mark up modifier entities related to the entity marked with <span> tag in the input text.

    ### Entity Markup Guide:
    Use <span class="uncertain"> to denote a measure of doubt.
    Use <span class="condition"> to denote a phrase that indicates the problems existing in a certain situation.
    Use <span class="subject"> to denote the person entity who is experiencing the disorder.
    Use <span class="negation"> to denote the phrase that indicates the absence of an entity.
    Use <span class="bodyloc"> to denote the location on the body where the observation is present.
    Use <span class="severity"> to denote the degree of intensity of a clinical condition.
    Use <span class="temporal"> to denote a calendar date, time, or duration related to a problem.
    Use <span class="course"> to denote the development or alteration of a problem.

    ### Input Text: {} <EOS>
    ### Output Text:'''
    
    treatment_prompt = '''### Task:
    Your task is to mark up modifier entities related to the entity marked with <span> tag in the input text.

    ### Entity Markup Guide:
    Use <span class="temporal"> to denote a calendar date, time, or duration related to a treatment.
    Use <span class="negation"> to denote the phrase that indicates the absence of an entity.

    ### Input Text: {} <EOS>
    ### Output Text:'''
    df = pd.DataFrame(columns=['main_entity','unprocessed', 'processed'])
    
    doc = nlp(text)
    sentences = list(doc.sents)
    sentence_relations = []

    unprocessed = []
    processed = []
    main_entities = []
    for sent in sentences:
        sent_relations = set()
        sent_entities = []

        # Check if entities are in the current sentence
        for ent_id, ent in entities.items():
            if ent[1] >= sent.start_char and ent[2] <= sent.end_char:
                sent_entities.append(ent)
                                
        # Check for existing relations in the current sentence
        for rel in relations:
            rel_type, ent1, ent2 = rel
            if ent1 in sent_entities and ent2 in sent_entities:
                sent_relations.add((ent1, ent2))
        sent_relations = group_relations(sent_relations)
        
        for main_entity, modifier_entities in sent_relations.items():
            #print (main_entity[0])
            main_entities.append(main_entity[0])
            # Replace entity mentions with their types
            modifier_sent_text = replace_entities_with_types(sent, modifier_entities).replace('\n',' ')
            main_entity_sent_text = replace_entities_with_types(sent, main_entity).replace('\n',' ')
            
            if main_entity[0] == 'problem':
                unprocessed.append(problem_prompt.format(main_entity_sent_text))
                processed.append(modifier_sent_text+' <EOS>')
            if main_entity[0] == 'drug':
                unprocessed.append(drug_prompt.format(main_entity_sent_text))
                processed.append(modifier_sent_text+' <EOS>')
            if main_entity[0] == 'treatment':
                unprocessed.append(treatment_prompt.format(main_entity_sent_text))
                processed.append(modifier_sent_text+' <EOS>')
            if main_entity[0] == 'test':
                unprocessed.append(test_prompt.format(main_entity_sent_text))
                processed.append(modifier_sent_text+' <EOS>')
        # check for non-existing relations in current sentence
        for entity in sent_entities:
            if entity not in sent_relations and entity[0] in ['problem','treatment','test','drug']:
                #print (entity[0])
                main_entities.append(entity[0])
                main_entity_sent_text = replace_entities_with_types(sent, entity).replace('\n',' ')
                sent_text = str(sent).replace('\n',' ')
                if entity[0] == 'problem':
                    unprocessed.append(problem_prompt.format(main_entity_sent_text))
                    processed.append(sent_text+' <EOS>')
                if entity[0] == 'drug':
                    unprocessed.append(drug_prompt.format(main_entity_sent_text))
                    processed.append(sent_text+' <EOS>')
                if entity[0] == 'treatment':
                    unprocessed.append(treatment_prompt.format(main_entity_sent_text))
                    processed.append(sent_text+' <EOS>')
                if entity[0] == 'test':
                    unprocessed.append(test_prompt.format(main_entity_sent_text))
                    processed.append(sent_text+' <EOS>')
    df = pd.concat([df, pd.DataFrame({'main_entity':main_entities,'unprocessed': unprocessed, 'processed': processed})], ignore_index=True)
    return df

def read_brat_files(txt_path, ann_path):
    with open(txt_path, "r",encoding='utf-8') as txt_file:
        text = txt_file.read()
    
    entities = {}
    relations = []
    
    new_lines = reorg_annfile(ann_path)
    for line in new_lines:
        try:
            parts = line.strip().split('\t')
            if parts[0].startswith('T'):
                try:
                    ent_id, ent_info = parts[0], parts[1]
                    ent_type, start, end = ent_info.split(' ')
                    ent_type = ent_type.lower()
                    entities[ent_id] = (ent_type, int(start), int(end))
                except:

                    print (new_lines)

                    raise
            elif parts[0].startswith('R'):
                rel_id, rel_type, arg1, arg2 = [parts[0]]+parts[1].split(' ')
                relations.append((rel_type, entities[arg1.split(':')[1]], entities[arg2.split(':')[1]]))
        except:
            continue
    return text, entities, relations

def follows_pattern(s):
    # Define the regex pattern
    pattern = r'^[TR]\d+'
    
    # Use re.match to check if the string matches the pattern
    if re.match(pattern, s):
        return True
    else:
        return False
    
def reorg_annfile(file):
    new_file=[]
    with open(file) as f:
        lines = f.readlines()
    skip = False
    for i,line in enumerate(lines):
        if i <= len(lines)-2:
            if not follows_pattern(lines[i+1]):
                new_file.append(line.strip()+' '+lines[i+1].strip()+'\n')
                skip = True
            else:
                if not skip:
                    new_file.append(line)
                else:
                    skip = False
                    pass
                
        else:
            if not skip:
                new_file.append(line)
            else:
                skip = False
                pass
    return new_file


In [6]:

separator = '\t'
batch_size = 100
for dataset in ['i2b2']:
    print(dataset)
    files = glob(f'../data/test/{dataset}/*.bio')

    prompts = []
    for i, file in enumerate(files):
        with open(file, 'r', encoding='utf-8') as f_read:
            text = ' '.join([line.split(separator)[0] for line in f_read.read().splitlines()])
        file_name = file.split('/')[-1].split('.')[0]
        prompts.append(prompt.format(text))

    prompts_list = batch_list(prompts, batch_size)

    outputs = []
    # Initialize tracking variables
    gpu_memory_usage_per_batch = []
    gpu_time_per_batch = []
    gpu_power_usage_per_batch = []

    # Start your batch processing loop
    for i, prompt_list in enumerate(prompts_list):
        start_time = time.time()

        # Record initial GPU power usage
        initial_power = get_gpu_power_usage()

        # Generate the output
        output = llm.generate(prompt_list, sampling_params, use_tqdm=False)

        # Record final GPU power usage
        final_power = get_gpu_power_usage()

        end_time = time.time()
        gpu_time = end_time - start_time
        gpu_time_per_batch.append(gpu_time)

        # Record GPU memory usage and power consumption after the batch
        gpu_memory_used = get_gpu_memory_usage()
        gpu_memory_usage_per_batch.append(gpu_memory_used)

        # Calculate average power usage for each GPU
        avg_power_usage = [(init + final) / 2 for init, final in zip(initial_power, final_power)]
        gpu_power_usage_per_batch.append(avg_power_usage)

        outputs += output

    # Calculate average GPU memory usage across all GPUs
    if gpu_memory_usage_per_batch:
        avg_gpu_memory_usage = [sum(mem) / len(gpu_memory_usage_per_batch) for mem in zip(*gpu_memory_usage_per_batch)]
        print(f'Average GPU memory usage (per GPU): {avg_gpu_memory_usage}')

    # Calculate total GPU time
    total_gpu_time = sum(gpu_time_per_batch)
    print(f'Total GPU time: {total_gpu_time} seconds (~{total_gpu_time / 3600:.2f} hours)')

    # Calculate average power consumption per GPU
    if gpu_power_usage_per_batch:
        avg_gpu_power_usage = [sum(power) / len(gpu_power_usage_per_batch) for power in zip(*gpu_power_usage_per_batch)]
        print(f'Average GPU power consumption (per GPU): {avg_gpu_power_usage}')

    # Calculate total energy consumption per GPU
    total_energy_consumption = [
        sum([power[i] * time / 3600 for power, time in zip(gpu_power_usage_per_batch, gpu_time_per_batch)])
        for i in range(len(avg_gpu_power_usage))
    ]
    print(f'Total energy consumption (per GPU): {total_energy_consumption} Wh')
    
    for i,seq in enumerate(outputs):
        file_name = files[i].split('/')[-1].split('.')[0]
        with open(f'./output/NER/{file_name}.html','w',encoding='utf-8') as f_write:
            f_write.write(seq.outputs[0].text)

i2b2
Average GPU memory usage (per GPU): [73329.25, 73329.25]
Total GPU time: 5.51078724861145 seconds (~0.00 hours)
Average GPU power consumption (per GPU): [294.42699999999996, 307.4886666666667]
Total energy consumption (per GPU): [0.4473378883051873, 0.46597867640925783] Wh


In [8]:

for dataset in ['i2b2_test']:
    # Load the SpaCy model for sentence tokenization
    nlp = spacy.load('en_core_web_lg')
    
    txt = glob(f'../data/RE/test_LLM/*.txt')

    relation_types = []
    entity_pairs = []
    df = pd.DataFrame(columns=['main_entity','unprocessed', 'processed'])
    for txt_path in txt:
        filename = txt_path.split('/')[-1].split('.')[0]
        ann_path = f'../data/RE/test_LLM/{filename}.ann'

        # Read the Brat files and extract the entities and relations
        text, entities, relations = read_brat_files(txt_path, ann_path)

        # Extract sentence relations
        df = pd.concat([df, sentence_relations(text, entities, relations, nlp)])
    
    print (len(df))
        
    unprocessed = []
    for index, row in df.iterrows():
        unprocessed.append(row['unprocessed'])
        

    batch_size = 100
    separator = '\t'
    
    prompts_list = batch_list(unprocessed, batch_size)

    outputs = []
    # Initialize tracking variables
    gpu_memory_usage_per_batch = []
    gpu_time_per_batch = []
    gpu_power_usage_per_batch = []

    # Start your batch processing loop
    for i, prompt_list in enumerate(prompts_list):
        start_time = time.time()

        # Record initial GPU power usage
        initial_power = get_gpu_power_usage()

        # Generate the output
        output = llm.generate(prompt_list, sampling_params, use_tqdm=False)

        # Record final GPU power usage
        final_power = get_gpu_power_usage()

        end_time = time.time()
        gpu_time = end_time - start_time
        gpu_time_per_batch.append(gpu_time)

        # Record GPU memory usage and power consumption after the batch
        gpu_memory_used = get_gpu_memory_usage()
        gpu_memory_usage_per_batch.append(gpu_memory_used)

        # Calculate average power usage for each GPU
        avg_power_usage = [(init + final) / 2 for init, final in zip(initial_power, final_power)]
        gpu_power_usage_per_batch.append(avg_power_usage)

        outputs += output

    # Calculate average GPU memory usage across all GPUs
    if gpu_memory_usage_per_batch:
        avg_gpu_memory_usage = [sum(mem) / len(gpu_memory_usage_per_batch) for mem in zip(*gpu_memory_usage_per_batch)]
        print(f'Average GPU memory usage (per GPU): {avg_gpu_memory_usage}')

    # Calculate total GPU time
    total_gpu_time = sum(gpu_time_per_batch)
    print(f'Total GPU time: {total_gpu_time} seconds (~{total_gpu_time / 3600:.2f} hours)')

    # Calculate average power consumption per GPU
    if gpu_power_usage_per_batch:
        avg_gpu_power_usage = [sum(power) / len(gpu_power_usage_per_batch) for power in zip(*gpu_power_usage_per_batch)]
        print(f'Average GPU power consumption (per GPU): {avg_gpu_power_usage}')

    # Calculate total energy consumption per GPU
    total_energy_consumption = [
        sum([power[i] * time / 3600 for power, time in zip(gpu_power_usage_per_batch, gpu_time_per_batch)])
        for i in range(len(avg_gpu_power_usage))
    ]
    print(f'Total energy consumption (per GPU): {total_energy_consumption} Wh')
    
    for i,seq in enumerate(outputs):
        with open(f'./output/RE/{i}.html','w',encoding='utf-8') as f_write:
            f_write.write(seq.outputs[0].text)

396
Average GPU memory usage (per GPU): [73329.25, 73329.25]
Total GPU time: 12.960270166397095 seconds (~0.00 hours)
Average GPU power consumption (per GPU): [324.594375, 339.533]
Total energy consumption (per GPU): [1.1455124690044256, 1.2499108208171528] Wh
